<div style="text-align: right" align="right"><i>Peter Norvig<br>Jan 2023</i></div>

# Docstring Fixpoint Theory

This notebook makes a proposal:

- One approach to writing a short function (in Python or other programming langauges) is to repeatedly alternate editing the **docstring** and the  **body** of the function until they converge to a **[fixpoint](https://en.wikipedia.org/wiki/Fixed_point)** in which there is an obvious one-to-one correspondance between the two.

This approach follows the first of [Tony Hoare](https://en.wikipedia.org/wiki/Tony_Hoare)'s two methods: *"There are two methods in software design. One is to make the program so simple, there are obviously no errors. The other is to make it so complicated, there are no obvious errors."* 

Some caveats: 
- This approach is not always appropriate! For many functions the docstring is a high-level description and the code has more detail that is not in the docstring. Docstring fixpoint theory makes the most sense  for very short functions.
- The edits will change the wording, but must maintain the meaning.

Python programs are easier to understand with [type hints](https://docs.python.org/3/library/typing.html), and dataclasses are nice, so I'll start with this:

In [1]:
from dataclasses import dataclass
from typing import *
Number = Union[int, float]

# Example: The Rainfall Problem

The "Rainfall Problem," initially posed by [Elliot Soloway](https://cdc.engin.umich.edu/elliot-soloway/), has been used to explore the ways that novices address a programming problem. We will use [Kathi Fisler](https://cs.brown.edu/~kfisler)'s [version](https://cs.brown.edu/~kfisler/Pubs/icer14-rainfall/) of the problem:


- *Design a program called **rainfall** that consumes a list of numbers representing daily rainfall amounts as entered by a user. The list may contain the number -999 indicating the end of the data of interest. Produce the average of the non-negative values in the list up to the first -999 (if it shows up). There may be negative numbers other than -999 in the list.* 

We start by writing a function prototype containing the complete problem statement as the **docstring**:

In [2]:
def rainfall(numbers: List[Number]) -> Number:
    """Design a program called rainfall that consumes a list of numbers 
    representing daily rainfall amounts as entered by a user. 
    The list may contain the number -999 indicating the end of the data of interest. 
    Produce the average of the non-negative values in the list up to the first -999 
    (if it shows up). There may be negative numbers other than -999 in the list."""
    ...

Now edit the **docstring** to remove extraneous parts, in the process abstracting the problem away from rainfall and focusing on the numbers involved:

In [3]:
def rainfall(numbers: List[Number]) -> Number:
    """Produce the average of the non-negative values in a list of numbers,
    up to the first -999 (if it shows up)."""
    ...

Next write a function **body** that mirrors the docstring as closely as possible. (It introduces helper functions.)

In [4]:
def rainfall(numbers: List[Number]) -> Number:
    """Produce the average of the non-negative values in a list of numbers,
    up to the first -999 (if it shows up)."""
    return mean(non_negative(upto(-999, numbers)))

Lightly edit the **docstring** once more to bring it into even closer agreement with the code:

In [5]:
def rainfall(numbers: List[Number]) -> Number:
    """Return the mean of the non-negative values in a list of numbers,
    up to the first -999 (if it shows up)."""
    return mean(non_negative(upto(-999, numbers)))

Fill in the missing bits, `mean`, `upto`, and `non_negative`:

In [6]:
from statistics import mean

def upto(end, items: list) -> list:
    """Return the list of items, but if `end` occurs in the list,
    only return the items that appear before `end`."""
    return items if end not in items else items[:items.index(end)]

def non_negative(numbers: List[Number]) -> List[Number]: 
    """The numbers that are greater than or equal to 0."""
    return [x for x in numbers if x >= 0]  

Write some tests and pass them, and we're done!

In [7]:
def test_rainfall():
    assert  0/2 == rainfall([0, 0]),               "no rain"
    assert  5/1 == rainfall([5]),                  "one day"
    assert  6/3 == rainfall([1, 2, 3]),            "the mean of several days"
    assert  6/4 == rainfall([0, 1, 2, 3]),         "the mean (which is a non-integer)"
    assert  6/4 == rainfall([1, 0, 3, 2]),         "order doesn't matter"
    assert  9/3 == rainfall([1, 2, -9, -100, 6]),  "negative values are ignored"
    assert  7/5 == rainfall([1, 0, 2, 0, 4]),      "zero values are not ignored"
    assert  8/3 == rainfall([1, 2, 5, -999, 404]), "values after -999 are ignored"
    return True
    
test_rainfall()

True

# More Complex Example: The Segments Problem

A [paper](https://www.cambridge.org/core/services/aop-cambridge-core/content/view/ABEA634EB9763953CBCC8D2AC58FE710/S0956796821000216a.pdf/segments-an-alternative-rainfall-problem.pdf) by [Peter Achten](https://www.ru.nl/en/people/achten-p) poses another problem:

- *Design a program called **segments** that consumes a list of numbers. Produce a list of all elements, without duplicates, and sorted in increasing order. Instead of containing all individual elements, these are organized as segments. A segment is either a single value x or a pair of two values a and b such that a, a+1, ...., b −1, b are in the list (neither a−1 nor b+1 are in the list). The segments must be shown as strings, formatted as x for singleton segments and a−b for the other segments.*

I'll start by taking the problem description verbatim, and breaking it up into two **docstrings**, one for the function `segments` that was requested in the description, and one for the class `Segment` that I think will be useful.

In [8]:
class Segment:
    """A segment is either a single value x or a pair of two values a and b such that 
    a, a+1, ...., b −1, b are in the list (neither a−1 nor b+1 are in the list). 
    The segments must be shown as strings, formatted as x for singleton segments 
    and a−b for the other segments."""
    ...
        
def segments(numbers: List[Number]) -> List[Segment]:
    """Design a program called segments that consumes a list of numbers. 
    Produce a list of all elements, without duplicates, and sorted in increasing order. 
    Instead of containing all individual elements, these are organized as segments."""
    ...

I'm not very happy with the docstring for `Segment`. First, the docstring  is mixing up two things: what a "segment" is, and how it relates to the numbers "in the list."  Second, `start` and `end` are better names than `a` and `b` for a segment.

Here's another try at the **docstring**:

In [9]:
class Segment:
    """`Segment(start, end)` represents the same sequence of integers as `range(start, end + 1)`,
    but a Segment is mutable. The `repr` is the string 'start-end', or just 'start' if start == end."""
    ...

Now I'll write the **body** for  `Segment`. It will be a dataclass with member variables `start` and `end`, abd a magic `repr` method. (A full `Segment` class should probably implement the same methods as `range`, but they are not needed for this problem.)

In [10]:
@dataclass
class Segment:
    """`Segment(start, end)` is like `range(start, end + 1)`, but is mutable and has fewer methods.
    `repr(Segment(start, end))` is the string 'start-end', or just 'start' if start == end."""
    start: int
    end: int
    
    def __repr__(self) -> str: 
        return f'{self.start}-{self.end}' if self.start != self.end else f'{self.start}'  

Let's make sure this works:

In [11]:
[Segment(7, 11), Segment(25, 25)]

[7-11, 25]

Now the **body** of the function `segments`: 

In [12]:
def segments(numbers: List[Number]) -> List[Segment]:
    """Design a program called `segments` that consumes a list of numbers. 
    Produce a list of all elements, without duplicates, and sorted in increasing order. 
    Instead of containing all individual elements, these are organized as segments."""
    return organize_as_segments(sorted(without_duplicates(numbers)))

This is pretty good, but I think it can be improved. The docstring has extraneous text ("Design a program") and could be more closely aligned with the body. And  it might be more modular to have a function to do one thing (add a single number to a list of segments), rather than a function that organizes all the numbers into a list of segments.  I'll edit the **docstring** first to reflect this change:

In [13]:
def segments(numbers: List[Number]) -> List[Segment]:
    """Return a segment list that covers all the `numbers`. Iterate through `numbers`
    (in sorted order and without duplicates) and add each number `n` to the evolving segment list."""
    ...

Now I'll edit the **body**:

In [14]:
def segments(numbers: List[Number]) -> List[Segment]:
    """Return a segment list that covers all the `numbers`. Iterate through `numbers`
    (in sorted order and without duplicates) and add each number `n` to the evolving segment list."""
    segment_list = []
    for n in sorted(without_duplicates(numbers)):
        add_to_segment_list(n, segment_list)
    return segment_list

There's not quite a one-to-one correspondance here between docstring and code. Maybe that means I'm following my advice of "This approach is not always appropriate!" Or maybe it means that the pattern of initializing a variable to the empty list is so well-known that I don't need to mention it in the docstring. 

In any case, here are the remaining missing pieces, `without_duplicates` and  `add_to_segment_list`:

In [15]:
without_duplicates = set # A set is a collection with no duplicates

def add_to_segment_list(n: int, segment_list: List[Segment]) -> None:
    """Mutate `segment_list` to cover `n`. If `n` is one more than the end of the last Segment, 
    then add `n` to that last segment. Otherwise append a new Segment (covering `n`) to `segment_list`."""
    if segment_list and segment_list[-1].end + 1 == n:
        segment_list[-1].end = n
    else:
        segment_list.append(Segment(n, n))   

Again, write some tests and pass them, and we're done!

In [16]:
def test_segments():
    assert segments([]) == [],                          "empty numbers"
    assert str(segments([42])) == '[42]',               "one number"
    assert str(segments([42, 42])) == '[42]',           "one number duplicated"
    assert str(segments([1, 2, 3])) == '[1-3]',         "one segment"
    assert str(segments([3, 1, 2])) == '[1-3]',         "same segment from unsorted input"
    numbers = [4, 0, 4, 8, 3, 1, 10, 2, 9, 7, 11, 24, 7]
    assert str(segments(numbers)) == '[0-4, 7-11, 24]', "multiple segments"
    for s in segments(numbers):
        assert type(s) == Segment and s.start <= s.end, "result is a list of valid Segments"
    assert without_duplicates([1, 2, 2, 1]) == {1, 2}, "no duplicates"

    s = []
    add_to_segment_list(1, s)
    assert s == [Segment(1, 1)]
    add_to_segment_list(2, s) 
    assert s == [Segment(1, 2)]
    add_to_segment_list(4, s)
    assert s == [Segment(1, 2), Segment(4, 4)]
    add_to_segment_list(5, s)
    assert s == [Segment(1, 2), Segment(4, 5)]
    
    return True

test_segments()

True